In [14]:
from jupyter_dash import JupyterDash
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import pandas as pd
import base64
import dash_leaflet as dl
import plotly.express as px


# Import your custom module
from aac_api import AnimalShelter

# Database connection
username = "aacuser"
password = "password"
db = AnimalShelter(username, password)

# Load initial data
df = pd.DataFrame.from_records(db.read({}))
df.drop(columns=['_id'], inplace=True)

# Encode the logo
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# App setup
app = JupyterDash(__name__)

app.layout = html.Div([
    html.Center(html.B(html.H1('Matt Krueger CS-340 Dashboard'))),
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={"height": "200px", "width": "200px"})),
    html.Hr(),
    html.Div(
        dcc.RadioItems(
            id='rescue-selector',
            options=[
                {"label": "Water Rescue", "value": "Water Rescue"},
                {"label": "Mountains Rescue", "value": "Mountains Rescue"},
                {"label": "Disaster Rescue", "value": "Disaster Rescue"},
                {"label": "Reset", "value": "Reset"}
            ],
            value="Reset",
            inline=True,
            style={'margin-bottom': '20px'}
        )
    ),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        row_selectable="single",
        selected_rows=[0],
        column_selectable="single",
        page_size=10,
        filter_action="native"
    ),
    html.Br(),
    html.Hr(),
    html.Div(
        className='row',
        style={'display': 'flex'},
        children=[
            html.Div(id='graph-id', className='col s12 m6'),
            html.Div(id='map-id', className='col s12 m6')
        ]
    )
])

@app.callback(
    Output('datatable-id', 'data'),
    [Input('rescue-selector', 'value')]
)
def filter_data(rescue_type):
    query = {}

    # Define MongoDB queries for each rescue type
    if rescue_type == "Water Rescue":
        query = {
            "breed": {
                "$in": [
                    "Labrador Retriever Mix", 
                    "Chesapeake Bay Retriever", 
                    "Newfoundland"
                ]
            },
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
    elif rescue_type == "Mountains Rescue":
        query = {
            "breed": {
                "$in": [
                    "German Shepherd", 
                    "Alaskan Malamute", 
                    "Old English Sheepdog", 
                    "Siberian Husky", 
                    "Rottweiler"
                ]
            },
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
    elif rescue_type == "Disaster Rescue":
        query = {
            "breed": {
                "$in": [
                    "Doberman Pinscher", 
                    "German Shepherd", 
                    "Golden Retriever", 
                    "Bloodhound", 
                    "Rottweiler"
                ]
            },
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
        }

    # Fetch all records
    elif rescue_type == "Reset":
        query = {}

    # Fetch and filter data based on query
    filtered_df = pd.DataFrame.from_records(db.read(query))
    filtered_df.drop(columns=['_id'], inplace=True)  # Cleanup MongoDB _id field
    return filtered_df.to_dict('records')

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    if viewData is None or index is None:
        return
    dff = pd.DataFrame.from_dict(viewData)
    row = index[0] if index else 0
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]], children=[
                dl.Tooltip(dff.iloc[row, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row, 9])
                ])
            ])
        ])
    ]


@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_pie_chart(viewData):
    if viewData is None or len(viewData) == 0:
        return dcc.Graph(figure={})  # Return empty chart if no data

    # Convert the data back to a DataFrame
    dff = pd.DataFrame.from_dict(viewData)

    # Create a pie chart for breed breakdown
    pie_chart = px.pie(
        dff,
        names='breed',
        title="Breed Breakdown",
        hole=0.3  # Optional: creates a donut chart effect
    )

    return dcc.Graph(figure=pie_chart)


app.run_server(debug=True)

Dash app running on http://127.0.0.1:10745/
